# Measuring the impact of student research labs at the University of Texas 

In [241]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import janitor 
import statsmodels.formula.api as smf
import censusdata 
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

In [274]:
df = pd.read_excel("IDA Student Info Fall 2018 - Test.xlsx", sheet_name= "Sheet1").clean_names()

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 10)
pd.set_option('display.min_rows', 10)
pd.set_option('display.max_colwidth', 25)


# Data Cleaning 

In [275]:
df = df.drop(['sch_maj2', 'ut_abbr2','cola_abbr2','maj2_name','cumulative_gpa_in_residenceunavailable_if_current_ccyys',
             'sem_gpa_fall_2018unavailable_if_current_ccyys','sem_gpa_hrs_fall_2018unavailable_if_current_ccyys',
             'unofficial_hnrs_pgm_s_','semester_hrs_incomplete_fall_2018_unavailable_if_current_ccyys','cumulative_hrs_incomplete', 'ut_degrees', 'cola_abbr1',
              'perm_street_3','perm_street_2','profiles'], axis=1)

df = df.rename(columns={"credit_hrs_excludes_current_": "credit_hrs_ex_cur", "current_ageas_of_today": "age", "cumulative_hrs_transferred" : "hrs_transferred"})



df['hs_rank'] = df['hs_rank'].astype(str)
df['overall_act_score'] = df['overall_act_score'].astype(str)
df['age'] = df['age'].astype(str)
df['sat_verbal_score'] = df['sat_verbal_score'].astype(str)
df['sat_quant_score'] = df['sat_quant_score'].astype(str)

df['age'] = df['age'].str.replace("\(R\)", "")
df['age'] = df['age'].str.replace("\*Minor", "")

df['classification'] = df['classification'].replace("\(R\)", "")
df = df.replace('unknown', "0/0")
df = df.replace('None', "0")

df['sat_verbal_score'] = df['sat_verbal_score'].astype(int)
df['sat_quant_score'] = df['sat_quant_score'].astype(int)
df['overall_act_score'] = df['overall_act_score'].astype(int)
df['age'] = df['age'].astype(int)

df[['numerator','denominator']] = df.hs_rank.str.split("/",expand=True,)
df['numerator'] = df['numerator'].astype(int)
df['denominator'] = df['denominator'].astype(int)
df['hs_rank'] = df['numerator']/df['denominator']
df['hs_rank'] = df['hs_rank']*100
df = df.drop(['numerator','denominator'], axis = 1)

df

,sch_maj1,ut_abbr1,maj1_name,cumulative_gpa,gpa_hrs,credit_hrs_ex_cur,current_hrs,perm_city,perm_state,perm_postal_code,citizenship,gender,ethnicities,classification,age,hrs_transferred,overall_act_score,sat_verbal_score,sat_quant_score,fatherseducation,motherseducation,familyincome,high_school,hs_rank
0,L 23400,C C,Classical Studies,3.1717,68,77,18,Lampasas,TX,76550-3612,U. S. Citizen,Male,Black or African Amer...,Junior,20,0,24,550,600,High School Diploma O...,Some College,0/0,Lampasas High School ...,6.048387
1,L 50600 (R),HIS,History,2.0767,43,76,17 (R),Austin,TX,78728-4418,U. S. Citizen,Female,Asian,Junior (R),21,35,23,560,590,Graduate Or Professio...,Graduate Or Professio...,H,Westwood High School ...,69.798658
2,L 00300,NaN,College of Liberal Ar...,4.0000,3,3,3,Austin,TX,78704-7957,U. S. Citizen,Female,Hispanic,Senior,32,0,0,0,0,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,C,Homer Hanna High Scho...,NaN
3,L 55000,IRG,International Relatio...,3.5208,34,79,15,Spring,TX,77388-5765,U. S. Citizen,Female,"Hispanic, Asian",Junior,20,42,22,660,540,Graduate Or Professio...,Associate's Degree,G,Klein Collins High Sc...,45.080946
4,L 47500,GOV,Government,0.0000,0,0,13,Lewisville,TX,75057-5123,U. S. Citizen,Female,Black or African Amer...,Freshman,18,0,0,700,600,Bachelor's Or Four-ye...,Educational Level Unk...,A,Lewisville High Schoo...,7.200720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10594,L 50600,HIS,History,2.4015,20,47,13,Edinburg,TX,78541-2287,U. S. Citizen,Female,"Hispanic, White",Sophomore,19,18,0,630,650,No High School,Associate's Degree,B,Edinburg North High S...,1.423488
10595,L 50600,HIS,History,4.0000,12,77,12,Austin,TX,78746-7806,U. S. Citizen,Female,White,Junior,19,45,31,0,0,Graduate Or Professio...,Bachelor's Or Four-ye...,I,Westlake High School ...,NaN
10596,L 55000,IRG,International Relatio...,2.8000,15,43,12,Austin,TX,78744-3644,U. S. Citizen,Female,Hispanic,Sophomore,19,0,24,0,0,Some High School,No High School,A,Kipp Austin Collegiat...,20.289855
10597,5 70700,ARH,Art History (Bachelor...,3.0738,81,112,18,Short Hills,NJ,070781653,U. S. Citizen,Female,White,Senior,21,22,0,600,570,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,G,Millburn High School ...,NaN


In [278]:
df.loc[df['ethnicities'] == 'Hispanic, Asian', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Black or African American','ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Asian, Black or African American, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Asian', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Black or African American, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, Black or African American, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian', 'ethnicities'] = "Mixed Race"

df

,sch_maj1,ut_abbr1,maj1_name,cumulative_gpa,gpa_hrs,credit_hrs_ex_cur,current_hrs,perm_city,perm_state,perm_postal_code,citizenship,gender,ethnicities,classification,age,hrs_transferred,overall_act_score,sat_verbal_score,sat_quant_score,fatherseducation,motherseducation,familyincome,high_school,hs_rank
0,L 23400,C C,Classical Studies,3.1717,68,77,18,Lampasas,TX,76550-3612,U. S. Citizen,Male,Black or African Amer...,Junior,20,0,24,550,600,High School Diploma O...,Some College,0/0,Lampasas High School ...,6.048387
1,L 50600 (R),HIS,History,2.0767,43,76,17 (R),Austin,TX,78728-4418,U. S. Citizen,Female,Asian,Junior (R),21,35,23,560,590,Graduate Or Professio...,Graduate Or Professio...,H,Westwood High School ...,69.798658
2,L 00300,NaN,College of Liberal Ar...,4.0000,3,3,3,Austin,TX,78704-7957,U. S. Citizen,Female,Hispanic,Senior,32,0,0,0,0,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,C,Homer Hanna High Scho...,NaN
3,L 55000,IRG,International Relatio...,3.5208,34,79,15,Spring,TX,77388-5765,U. S. Citizen,Female,Mixed Race,Junior,20,42,22,660,540,Graduate Or Professio...,Associate's Degree,G,Klein Collins High Sc...,45.080946
4,L 47500,GOV,Government,0.0000,0,0,13,Lewisville,TX,75057-5123,U. S. Citizen,Female,Black or African Amer...,Freshman,18,0,0,700,600,Bachelor's Or Four-ye...,Educational Level Unk...,A,Lewisville High Schoo...,7.200720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10594,L 50600,HIS,History,2.4015,20,47,13,Edinburg,TX,78541-2287,U. S. Citizen,Female,Mixed Race,Sophomore,19,18,0,630,650,No High School,Associate's Degree,B,Edinburg North High S...,1.423488
10595,L 50600,HIS,History,4.0000,12,77,12,Austin,TX,78746-7806,U. S. Citizen,Female,White,Junior,19,45,31,0,0,Graduate Or Professio...,Bachelor's Or Four-ye...,I,Westlake High School ...,NaN
10596,L 55000,IRG,International Relatio...,2.8000,15,43,12,Austin,TX,78744-3644,U. S. Citizen,Female,Hispanic,Sophomore,19,0,24,0,0,Some High School,No High School,A,Kipp Austin Collegiat...,20.289855
10597,5 70700,ARH,Art History (Bachelor...,3.0738,81,112,18,Short Hills,NJ,070781653,U. S. Citizen,Female,White,Senior,21,22,0,600,570,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,G,Millburn High School ...,NaN


### Downloading Census Data 

In [30]:
urban_areas = censusdata.download('acs5', 2015, censusdata.censusgeo([('urban area', '*')]),
                                   ['B01001_001E'])

urban_areas 

,B01001_001E
"Columbus, IN Urbanized Area (2010): Summary level: 400, urban area:19126",56902
"Columbus, KS Urban Cluster (2010): Summary level: 400, urban area:19153",3342
"Columbus, MS Urban Cluster (2010): Summary level: 400, urban area:19180",30529
"Columbus, NE Urban Cluster (2010): Summary level: 400, urban area:19207",22404
"Columbus, OH Urbanized Area (2010): Summary level: 400, urban area:19234",1430416
...,...
"Thomasville, GA Urban Cluster (2010): Summary level: 400, urban area:87436",25790
"Thomson, GA Urban Cluster (2010): Summary level: 400, urban area:87463",8618
"Thousand Oaks, CA Urbanized Area (2010): Summary level: 400, urban area:87490",220357
"Three Rivers, MI Urban Cluster (2010): Summary level: 400, urban area:87517",10651


In [33]:
test = pd.read_csv('usa_00001.dat', sep='\s+', index_col=0,
                 header=None)
test

C:\Users\tenis\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,1
0,
2018,201801000000012018010000049000000750020180000000110100044622000000000000000016000140001000000750...
2018,201801000000022018010000058000000750020180000000210108141222000000000000000019000140001000000750...
2018,201801000000032018010000219000001180020180000000310100043386000000000000000020000130001000001180...
2018,201801000000042018010000246000000430020180000000410100000000000000000000000024000130001000000430...
2018,201801000000052018010000251000000160020180000000510109743366000000000000000027010130001000000160...
...,...
2018201820180306924393201800140032600000021002018069243933560000000000000,000000040056100040000001700009
2018201820180306924393201800140032600000021002018069243933560000000000000,000000040056100050000001700007
2018201820180306924394201800140050200000008002018069243943560001000000000,000000010056100010000000900049


In [34]:
from xml.dom import minidom


def row_generator(datapath, ddipath):
    ''' Maps each line of the data file to the variables and values
        it represents '''
    # get mapping
    pmap = pos_map(ddipath)
    f = open(datapath, 'r')
    for line in f:
        # apply mapping
        row = {}
        for var in pmap.keys():
            start = pmap[var]['spos']
            end = pmap[var]['epos']
            dec = pmap[var]['dec']
            if dec:
                mid = end - dec
                row[var] = line[start:mid] + '.' + line[mid:end]
            else:
                row[var] = line[start : end]
        # yield mapping
        yield row


def pos_map(ddipath):
    ''' Returns a dictionary mapping the variable names to their positions
        and decimal places in the data file '''
    m = minidom.parse(ddipath)    
    vmap = {}
    varNodes = m.getElementsByTagName('var')
    for varNode in varNodes:
        locNode = varNode.getElementsByTagName('location')[0]
        name = varNode.attributes.getNamedItem('ID').value
        vmap[name] = {
            'spos' : int(locNode.attributes.getNamedItem('StartPos').value) - 1,
            'epos' : int(locNode.attributes.getNamedItem('EndPos').value),
            'dec' : int(varNode.attributes.getNamedItem('dcml').value)
            }
    return vmap

rows = row_generator(datapath = "usa_00001.dat", ddipath = "usa_00001.xml")


# Statistical Analysis 

In [2]:
#model = smf.ols("cumulative_gpa ~ citizenship + age + sat_verbal_score + sat_quant_score + hs_rank + ut_abbr1 + motherseducation + fatherseducation + ethnicities", df)
#results = model.fit()
#results.summary()